<a href="https://colab.research.google.com/github/HoYoungChun/financial_data_analysis/blob/main/%EB%B3%80%EB%8F%99%EC%84%B1_%ED%83%80%EA%B2%8C%ED%8C%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 변동성 타게팅(Volatility Targeting)

### Load Libraries & Dataset

In [1]:
import pandas as pd
import requests
from tqdm import tqdm

#### KODEX 200

In [2]:
kodex_200 = pd.DataFrame()

kodex_url = 'https://finance.naver.com/item/sise_day.nhn?code=069500'

for page in tqdm(range(1,100)):
    pg_url = '{url}&page={page}'.format(url=kodex_url, page=page)
    pg_url = requests.get(pg_url, headers={'User-agent':'Mozilla/5.0'}).text
    kodex_200 = kodex_200.append(pd.read_html(pg_url)[0], ignore_index=True)
    #pd.read_html(pg_url)[0]
kodex_200

100%|██████████| 99/99 [01:03<00:00,  1.55it/s]


,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.07.15,43545.0,145.0,43525.0,43555.0,43380.0,573607.0
2,2021.07.14,43400.0,150.0,43400.0,43465.0,43225.0,3453656.0
3,2021.07.13,43550.0,365.0,43380.0,43565.0,43380.0,3654600.0
4,2021.07.12,43185.0,385.0,43060.0,43260.0,43060.0,3929269.0
...,...,...,...,...,...,...,...
1480,2017.07.11,31480.0,175.0,31345.0,31500.0,31300.0,4983074.0
1481,2017.07.10,31305.0,105.0,31300.0,31390.0,31220.0,4338365.0
1482,2017.07.07,31200.0,125.0,31200.0,31285.0,31115.0,5436532.0
1483,2017.07.06,31325.0,5.0,31395.0,31395.0,31240.0,6394300.0


In [3]:
kodex_200 = kodex_200.dropna()
kodex_200.columns = ['date','close','over_the_day','open','high','low','volume']
kodex_200.date = pd.to_datetime(kodex_200.date)

print('출처: ', kodex_url)
print(kodex_200.shape)
kodex_200.head()

출처:  https://finance.naver.com/item/sise_day.nhn?code=069500
(990, 7)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,date,close,over_the_day,open,high,low,volume
1,2021-07-15,43545.0,145.0,43525.0,43555.0,43380.0,573607.0
2,2021-07-14,43400.0,150.0,43400.0,43465.0,43225.0,3453656.0
3,2021-07-13,43550.0,365.0,43380.0,43565.0,43380.0,3654600.0
4,2021-07-12,43185.0,385.0,43060.0,43260.0,43060.0,3929269.0
5,2021-07-09,42800.0,460.0,42840.0,43050.0,42445.0,8288519.0


In [4]:
kodex_200 = kodex_200.sort_values(by='date')
kodex_200.index = [x for x in range(len(kodex_200))]
kodex_200

,date,close,over_the_day,open,high,low,volume
0,2017-07-06,31325.0,5.0,31395.0,31395.0,31240.0,6394300.0
1,2017-07-07,31200.0,125.0,31200.0,31285.0,31115.0,5436532.0
2,2017-07-10,31305.0,105.0,31300.0,31390.0,31220.0,4338365.0
3,2017-07-11,31480.0,175.0,31345.0,31500.0,31300.0,4983074.0
4,2017-07-12,31475.0,5.0,31480.0,31555.0,31450.0,9780095.0
...,...,...,...,...,...,...,...
985,2021-07-09,42800.0,460.0,42840.0,43050.0,42445.0,8288519.0
986,2021-07-12,43185.0,385.0,43060.0,43260.0,43060.0,3929269.0
987,2021-07-13,43550.0,365.0,43380.0,43565.0,43380.0,3654600.0
988,2021-07-14,43400.0,150.0,43400.0,43465.0,43225.0,3453656.0


#### Convert into Monthly data

In [5]:
kodex_monthly = pd.DataFrame()

for y in range(2017,2022):
    for m in range(1,13):
        kodex_monthly = kodex_monthly.append(kodex_200[(kodex_200['date'].dt.year == y) & (kodex_200['date'].dt.month==m)].head(1))

In [ ]:
# list comprehension
kodex_monthly.index = [x for x in range(len(kodex_monthly))]
kodex_monthly

## Calculate Volatility(Standard deviation)

In [ ]:
std_list = []
for y in range(2017,2022):
    for m in range(1,13):
        abs_std = kodex_200[(kodex_200.date.dt.year==y) & (kodex_200.date.dt.month == m)].std().close
        monthly_mean = kodex_200[(kodex_200.date.dt.year==y) & (kodex_200.date.dt.month == m)].mean().close
        rel_std = abs_std / monthly_mean * 100

        if rel_std > 0:
            std_list.append(rel_std)

#### Calculate RoR

In [16]:
ror_list=[]
for i in range(len(kodex_monthly)):
    try:
        ror = kodex_monthly.close.diff().iloc[i] / kodex_monthly.close.iloc[i-1] * 100
    except:
        ror = 0
    ror_list.append(ror)

In [22]:
kodex_monthly['RoR'] = ror_list
kodex_monthly = kodex_monthly.fillna(0)
kodex_monthly.head()

,date,close,over_the_day,open,high,low,volume,RoR,Plust,Plus
0,2017-07-06,31325.0,5.0,31395.0,31395.0,31240.0,6394300.0,0.000000,False,False
1,2017-08-01,31740.0,280.0,31395.0,31905.0,31355.0,8635707.0,1.324820,True,True
2,2017-09-01,30810.0,0.0,30915.0,30955.0,30700.0,8286500.0,-2.930057,False,False
3,2017-10-10,32275.0,575.0,32185.0,32450.0,32105.0,11924669.0,4.754950,True,True
4,2017-11-01,33925.0,550.0,33665.0,33925.0,33575.0,14991272.0,5.112316,True,True


In [23]:
kodex_monthly['Plus'] = kodex_monthly.RoR > 0
kodex_monthly.head()

,date,close,over_the_day,open,high,low,volume,RoR,Plust,Plus
0,2017-07-06,31325.0,5.0,31395.0,31395.0,31240.0,6394300.0,0.000000,False,False
1,2017-08-01,31740.0,280.0,31395.0,31905.0,31355.0,8635707.0,1.324820,True,True
2,2017-09-01,30810.0,0.0,30915.0,30955.0,30700.0,8286500.0,-2.930057,False,False
3,2017-10-10,32275.0,575.0,32185.0,32450.0,32105.0,11924669.0,4.754950,True,True
4,2017-11-01,33925.0,550.0,33665.0,33925.0,33575.0,14991272.0,5.112316,True,True


In [ ]:
kodex_monthly.Plus.rolling(window=12).sum()
kodex_monthly.head(20)

In [ ]:
kodex_monthly = kodex_monthly.fillna(0)
kodex_monthly